Step 1: Setting Up Google Colab Environment

In [2]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install torch torchvision
!pip install gdown

from google.colab import drive
import os
import zipfile
import glob
import yaml
import shutil
import random
from ultralytics import YOLO
import torch

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 52.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


Step 2: Loading the Dataset from Google Drive

In [3]:
import os
import zipfile

dataset_zip_path = '/content/drive/MyDrive/final_dataset_v1.zip'  # Update this path

with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/HPD_dataset')


Step 3: Preparing YOLOv8 for Training

In [5]:
import yaml

# Creating a custom YAML configuration file for YOLO
data_yaml = {
    'path': '/content/HPD_dataset/final_dataset_v1',
    'train': 'train',
    'val': 'val',
    'nc': 5,
    'names': ['Five_VFR', 'Four_VFR', 'One_VFR', 'Three_VFR', 'Two_VFR']
}

with open('/content/HPD_dataset/data.yaml', 'w') as file:
    yaml.dump(data_yaml, file)


Making validation data

In [6]:
import os
import shutil
import random

dataset_path = '/content/HPD_dataset/final_dataset_v1'

# Create 'val' directory if it doesn't exist
val_dir = os.path.join(dataset_path, 'val')
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)


# Move a portion of data from 'train' to 'val'
train_images_dir = os.path.join(dataset_path, 'train', 'images')
train_labels_dir = os.path.join(dataset_path, 'train', 'labels')
val_images_dir = os.path.join(val_dir, 'images')
val_labels_dir = os.path.join(val_dir, 'labels')

all_images = os.listdir(train_images_dir)
num_images = len(all_images)
num_val_images = int(0.1 * num_images)  # 20% for validation

# Randomly select images for validation
val_images = random.sample(all_images, num_val_images)

# Move selected images and labels
for image_name in val_images:
    # Move image
    source_image_path = os.path.join(train_images_dir, image_name)
    destination_image_path = os.path.join(val_images_dir, image_name)
    shutil.move(source_image_path, destination_image_path)

    # Move label (if it exists)
    label_name = os.path.splitext(image_name)[0] + '.txt'
    source_label_path = os.path.join(train_labels_dir, label_name)
    destination_label_path = os.path.join(val_labels_dir, label_name)
    if os.path.exists(source_label_path):
        shutil.move(source_label_path, destination_label_path)

Step 4: Fine-Tuning YOLOv8 Model

In [7]:
# Load YOLO
model = YOLO('yolov8n.pt')

# Define the directory to save checkpoints in Google Drive
checkpoint_dir = '/content/drive/MyDrive/YOLOv8_checkpoints'

# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

# Training loop with manual saving
for epoch in range(100):
    # Train for one epoch
    results = model.train(data='/content/HPD_dataset/data.yaml', epochs=1, imgsz=640)

    # Save the model weights
    if (epoch + 1) % 5 == 0:
      save_path = os.path.join(checkpoint_dir, f'yolov8n_epoch_{epoch + 1}.pt')
      torch.save(model.model.state_dict(), save_path)
      print(f"Model saved to: {save_path}")


100%|██████████| 6.25M/6.25M [00:00<00:00, 244MB/s]


Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HPD_dataset/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 755k/755k [00:00<00:00, 65.5MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 238MB/s]


KeyboardInterrupt: 

Step 5: Resuming Training from Checkpoints (uncomment when disconnected)

In [10]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt') # Load the base model architecture

# Load the saved weights, addressing the key mismatch
# Get the state dictionary from the saved checkpoint
state_dict = torch.load('/content/drive/MyDrive/YOLOv8_checkpoints/yolov8n_epoch_95.pt')

# Modify keys to match the expected structure (remove 'model.')
modified_state_dict = {k[6:]: v for k, v in state_dict.items() if k.startswith('model.')}

# Load the modified state dictionary
model.load_state_dict(modified_state_dict, strict=False)

# Continue training from the loaded checkpoint
for epoch in range(95, 100):  # Assuming you want to train for 100 epochs in total
    results = model.train(data='/content/HPD_dataset/data.yaml', epochs=1, imgsz=640)

    # Save the model weights
    if (epoch + 1) % 5 == 0:
        save_path = os.path.join(checkpoint_dir, f'yolov8n_epoch_{epoch + 1}.pt')
        torch.save(model.model.state_dict(), save_path)
        print(f"Model saved to: {save_path}")

Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/HPD_dataset/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning /content/HPD_dataset/final_dataset_v1/train/labels.cache... 489 images, 0 backgrounds, 0 corrupt: 100%|██████████| 489/489 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/HPD_dataset/final_dataset_v1/val/labels.cache... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Plotting labels to runs/detect/train32222/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train32222
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1       2.3G     0.9682      1.344      1.015         31        640: 100%|██████████| 31/31 [00:20<00:00,  1.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.75it/s]


                   all         54        101      0.618      0.724      0.693      0.462

1 epochs completed in 0.014 hours.
Optimizer stripped from runs/detect/train32222/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train32222/weights/best.pt, 6.2MB

Validating runs/detect/train32222/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.35it/s]


                   all         54        101      0.619      0.727      0.694      0.462
              Five_VFR          8         16      0.897        0.5      0.774      0.531
              Four_VFR          8         16      0.579      0.938      0.743      0.599
               One_VFR         12         21      0.445       0.81      0.456      0.283
             Three_VFR          5         10       0.49      0.386      0.516      0.314
               Two_VFR         21         38      0.686          1      0.979      0.584
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train32222
Model saved to: /content/drive/MyDrive/YOLOv8_checkpoints/yolov8n_epoch_100.pt


In [11]:
#saveing the model
model.save('/content/drive/MyDrive/YOLOv8_final_version1')

Step 6: Testing the Trained Model on a Video

In [12]:
import cv2

# Path to the input video
video_path = '/content/drive/MyDrive/All.mp4'  # Update this path

# Path to the output video
output_video_path = '/content/drive/MyDrive/output_video_one_vfr.mp4'  # Update this path

# Load the trained model
model = YOLO('/content/drive/MyDrive/YOLOv8_final_version1')

# Open the input video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Draw bounding boxes and labels on the frame
    for *xyxy, conf, cls in results[0].boxes.data:  # Access bounding boxes through 'boxes' and its data
        x1, y1, x2, y2 = map(int, xyxy)  # Extract coordinates from xyxy
        label = model.names[int(cls)]  # Get label from class ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()
print("Processed video saved at:", output_video_path)


Streaming output truncated to the last 5000 lines.

0: 288x640 1 Four_VFR, 1 One_VFR, 1 Two_VFR, 10.9ms
Speed: 2.9ms preprocess, 10.9ms inference, 7.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 One_VFR, 1 Two_VFR, 18.6ms
Speed: 2.6ms preprocess, 18.6ms inference, 2.8ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 One_VFR, 1 Two_VFR, 21.5ms
Speed: 5.0ms preprocess, 21.5ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 One_VFR, 1 Two_VFR, 17.8ms
Speed: 4.4ms preprocess, 17.8ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 One_VFR, 1 Two_VFR, 14.6ms
Speed: 3.4ms preprocess, 14.6ms inference, 2.3ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 Two_VFR, 12.2ms
Speed: 3.5ms preprocess, 12.2ms inference, 4.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 Four_VFR, 1 Two_VFR, 11.2ms
Speed: 4.4ms prepr